In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
import datetime
import torchvision
from torchvision.datasets import MNIST
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class Model1(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(nn.Conv2d(1, 8, (3,3), stride = 1, padding = 'same'),    #28x28x8
                               nn.Conv2d(8, 8, (3, 3), stride = 1, padding = 'same'),   #28x28x8
                               nn.MaxPool2d((3, 3), stride = 2, padding = 1),      #15x15x8
                               nn.Conv2d(8, 4, (1, 1), stride = 1, padding = 'same' ),  #15x15x4
                               nn.Conv2d(4, 4, (1, 1), stride = 1, padding = 'same' ),  #15x15x4
                              #  nn.MaxPool2d((3, 3), stride = 2, padding = 1),      #15x15x8
                               nn.ReLU(), 
                               nn.Flatten(),    
                               nn.Linear(784, 400),
                              #  nn.Linear(196, 100),
                               nn.Linear(400, 100),
                               nn.Linear(100, 10))
    self.train_data = MNIST('data', train = True, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
    self.test_data = MNIST('data', train = False, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
    # print(type(self.datasets.train_data))
  def forward(self, x):
    return nn.Softmax(dim=1)(self.model(x))


In [4]:
class Model2(Model1):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(nn.Conv2d(1, 8, (3,3), stride = 1, padding = 'same'),    #28x28x8
                               nn.Conv2d(8, 8, (3, 3), stride = 1, padding = 'same'),   #28x28x8
                               nn.Conv2d(8, 8, (3,3), stride = 1, padding='same'),
                               nn.MaxPool2d((3, 3), stride = 2, padding = 1),      #15x15x8
                               nn.Conv2d(8, 4, (1, 1), stride = 1, padding = 'same' ),  #15x15x4
                               nn.Conv2d(4, 4, (1, 1), stride = 1, padding = 'same' ),  #15x15x4
                               nn.Conv2d(4, 2, (1, 1), stride = 1, padding = 'same' ),  #15x15x4
                               nn.ReLU(), 
                               nn.Flatten(),    #
                               nn.Linear(392, 100),
                               nn.Linear(100, 10))
    self.train_data = MNIST('data', train = True, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
    self.test_data = MNIST('data', train = False, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))



In [5]:
class Model3(Model1):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(nn.Conv2d(1, 8, (3,3), stride = 1, padding = 'same'),    #28x28x8
                               nn.Conv2d(8, 8, (3, 3), stride = 1, padding = 'same'),   #28x28x8
                               nn.Conv2d(8, 8, (3,3), stride = 1, padding='same'),
                               nn.MaxPool2d((3, 3), stride = 2, padding = 1),      #15x15x8
                               nn.Conv2d(8, 4, (3,3), stride = 1, padding = 'same'), #15x15x6
                              #  nn.Conv2d(4, 4, (1, 1), stride = 1, padding = 'same' ),  #15x15x4
                               nn.ReLU(), 
                               nn.Flatten(),    #
                               nn.Linear(784, 400),
                               nn.Linear(400, 100),
                               nn.Linear(100, 10))
    self.train_data = MNIST('data', train = True, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))
    self.test_data = MNIST('data', train = False, download=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]))



In [6]:
def model_state_dict(path):
  from collections import OrderedDict
  new_state_dict = OrderedDict()
  state_dict = torch.load(path)
  for k, v in state_dict.items():
      # print(k)
      # print(v)
      name = 'model.' + k  # add `model.`
      # print(name)
      new_state_dict[name] = v
  return new_state_dict

In [7]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


In [8]:
class Adv():
  def __init__(self, model_dict):
    super().__init__()
    self.model = {}
    self.loss_fn = nn.CrossEntropyLoss()
    
    for key, value in model_dict.items():
      # print(key)
      self.model[key] = value[0]
      self.model[key].load_state_dict(value[1])
      self.model[key].to(device)
      self.model[key].eval()
    # print(self.model)

  def pred(self, x, y):
    ypred = {}
    loss = 0
    for key, value in self.model.items():
      
      ypred[key] = value.model(x)
      _, ypred[key] = torch.max(value.model(x), 1)
      print(key, value.model, ypred[key].dtype)
      loss += self.loss_fn(y, value.model(x)) 
    return ypred, loss

  def linbp_relu(self, x):
    x_p = F.relu(-x)
    x = x + x_p.data
    return x

  def model_fwd(self, model, x, linbp, linbp_layer):
    # x = model[0](x)
    for idx in range(len(list(model))):
        if linbp and isinstance(model[idx], nn.ReLU) and idx >= linbp_layer:
            x = self.linbp_relu(x)
        else:
            x = model[idx](x)
    x = x.view(x.size(0), -1)
    # x = model[1].classifier(x)
    return x


    
    


In [9]:
Path1 = "/content/drive/MyDrive/CS726_project/MNIST_ENSEMBLE_MODEL_1/2022-04-06 21:21/epoch_20.pth"
Path2 = "/content/drive/MyDrive/CS726_project/MNIST_ENSEMBLE_MODEL_2/2022-04-06 21:37/epoch_20.pth"
Path3 = "/content/drive/MyDrive/CS726_project/MNIST_ENSEMBLE_MODEL_3/2022-04-07 06:38/epoch_20.pth"

In [17]:
model1 = Model1()
state_dict1 = model_state_dict(Path1) 

model2 = Model2()
state_dict2 = model_state_dict(Path2) 

model3 = Model3()
state_dict3 = model_state_dict(Path3)

model_dict = {'model1': (model1, state_dict1), 'model2': (model2, state_dict2), 'model3': (model3, state_dict3) }
# model_dict = {'model1': (model1, state_dict1), 'model2': (model2, state_dict2)}

adv = Adv(model_dict)



In [18]:
train_dl = DataLoader(model1.train_data, batch_size = 1)

# print(train_dl[0])
for idx, data in enumerate(train_dl):
  ori_img = data[0]
  label = data[1]


In [19]:
output = adv.model_fwd(adv.model['model1'].model, ori_img, True, 2)

In [20]:
target_attack = True
model = adv.model['model1'].model
epsilon = 1e-3

In [21]:
img = ori_img.clone()

In [22]:
loss = nn.CrossEntropyLoss()(output, label.to(device))
model.zero_grad()
loss.backward(retain_graph=True)
img.requires_grad_(True)
g = img.grad.data
m = 0
iters = 5
for i in range(iters):
  input_grad = 1 * m + g / torch.norm(g, dim=(1, 2, 3), p=1, keepdim=True)
  m = input_grad
if target_attack:
  input_grad = -input_grad
img = torch.where(img > ori_img + epsilon, ori_img + epsilon, img)
img = torch.where(img < ori_img - epsilon, ori_img - epsilon, img)
img = torch.clamp(img, min=0, max=1)


AttributeError: ignored

In [23]:
pred, loss = adv.pred(img, label)
print(pred)
print(label)
print(loss)

model1 Sequential(
  (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (1): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (2): MaxPool2d(kernel_size=(3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (3): Conv2d(8, 4, kernel_size=(1, 1), stride=(1, 1), padding=same)
  (4): Conv2d(4, 4, kernel_size=(1, 1), stride=(1, 1), padding=same)
  (5): ReLU()
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=784, out_features=400, bias=True)
  (8): Linear(in_features=400, out_features=100, bias=True)
  (9): Linear(in_features=100, out_features=10, bias=True)
) torch.int64


RuntimeError: ignored

In [ ]:
model_dummy = Model1()
# dummy_state_dict = model_state_dict(Path1)
model_dummy.model.load_state_dict(torch.load(Path1))

<All keys matched successfully>

In [ ]:
output = model_dummy.model(img)

In [ ]:
print(output)

tensor([[ 1.8050, -3.1402, -0.5270, -0.8103, -2.9675,  1.8368, -3.2570, -0.5435,
          5.6767,  2.3905]], grad_fn=<AddmmBackward0>)


In [25]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")

94781440/94765736 [==============================] - 1s 0us/step
